In [77]:
from urllib.request import urlopen as uReq
import requests
import bs4 as bs


url = "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=AAPL&type=&dateb=&owner=exclude&start=0&count=80&output=atom"
headers = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36"}
download = requests.get(f'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=0000320193&type=&dateb=&owner=exclude&start=0&count=80&output=atom',headers=headers).content
#download=download.decode("utf-8").split('\n')

html = bs.BeautifulSoup(download, 'xml')
#print(html.prettify())
entries = html.findAll("entry")
    
shouldContinue = True
link = ""

print(link)
for entry in entries:
    #print(entry)
    if shouldContinue and (entry.find("category")["term"].lower() == "10-k" or entry.find("category")["term"].lower() == "10-q" or entry.find("category")["term"].lower() == "20-f"):
        
        firstUrl = entry.find("link")["href"]
        uClientFirstUrl = requests.get(firstUrl,headers=headers).content
        htmlFirstUrl = bs.BeautifulSoup(uClientFirstUrl, 'xml')
        #print(htmlFirstUrl.prettify())
        tds = htmlFirstUrl.findAll("table")
        #print(tds)
        tdresult = tds[1].findAll("td")
        print(tdresult)
        foundtd = False
        for td in tdresult:
            if foundtd == True:
                link = "https://www.sec.gov" + td.find("a")["href"]
                print('Link:')
                print(link)
                foundtd = False
            if "xbrl instance" in td.text.lower():
                foundtd = True

            shouldContinue = False
    #print(link)



[<td scope="row">5</td>, <td scope="row">XBRL TAXONOMY EXTENSION SCHEMA DOCUMENT</td>, <td scope="row"><a href="/Archives/edgar/data/320193/000032019322000059/aapl-20220326.xsd">aapl-20220326.xsd</a></td>, <td scope="row">EX-101.SCH</td>, <td scope="row">35163</td>, <td scope="row">6</td>, <td scope="row">XBRL TAXONOMY EXTENSION CALCULATION LINKBASE DOCUMENT</td>, <td scope="row"><a href="/Archives/edgar/data/320193/000032019322000059/aapl-20220326_cal.xml">aapl-20220326_cal.xml</a></td>, <td scope="row">EX-101.CAL</td>, <td scope="row">85165</td>, <td scope="row">7</td>, <td scope="row">XBRL TAXONOMY EXTENSION DEFINITION LINKBASE DOCUMENT</td>, <td scope="row"><a href="/Archives/edgar/data/320193/000032019322000059/aapl-20220326_def.xml">aapl-20220326_def.xml</a></td>, <td scope="row">EX-101.DEF</td>, <td scope="row">157188</td>, <td scope="row">8</td>, <td scope="row">XBRL TAXONOMY EXTENSION LABEL LINKBASE DOCUMENT</td>, <td scope="row"><a href="/Archives/edgar/data/320193/000032019

In [89]:
import logging
from xbrl.cache import HttpCache
from xbrl.instance import XbrlInstance, XbrlParser

logging.basicConfig(level=logging.INFO)
cache: HttpCache = HttpCache('./cache')
# Replace the dummy header with your information!! 
# services like SEC EDGAR require you to disclose information about your bot! (https://www.sec.gov/privacy.htm#security)
cache.set_headers({'From': 'your.name@company.com', 'User-Agent': 'Tool/Version (Website)'})
xbrlParser = XbrlParser(cache)
 
xbrl_url = link
inst: XbrlInstance = xbrlParser.parse_instance(xbrl_url)





In [90]:
import pandas as pd
import re
extracted_data: [dict] = []
selected_facts: [str] = ['Assets', 'Liabilities', 'StockholdersEquity','SalesRevenueNet']

# for fact in inst.facts:
#     p
#     print(fact.concept.name)
    
for fact in inst.facts:
    # use some kind of filter, otherwise your dataframe will have maaaaannnyyy columns (one for every concept)
    #if fact.concept.name not in selected_facts: continue
    # only select non-dimensional data for now
    if len(fact.context.segments) > 0: continue
    if hasattr(fact.context,'instant_date') and (fact,'value') and (fact.concept,'name')  :
        concept_name=re.split('(?=[A-Z])', fact.concept.name)
        name=''
        for word in concept_name:
            name+=' '+word

        extracted_data.append({'date': fact.context.instant_date, 'concept': name , 'value': fact.value})



df: pd.DataFrame = pd.DataFrame(data=extracted_data)
df.drop_duplicates(inplace=True)
#pivot the dataframe so that the concept name is now the column
pivot_df: pd.DataFrame() = df.pivot(index='date', columns='concept')
df.to_csv('parsed.csv', sep='\t', encoding='utf-8')

ValueError: Index contains duplicate entries, cannot reshape